# Guidelines for Prompt Engineering

## Setup

https://platform.openai.com/settings/organization/usage

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

key = os.getenv('OPEN_API_KEY')
client = openai.OpenAI(api_key=key)

# helper function
def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7  # Adjust creativity
        )
    return response.choices[0].message.content

*Prompting Principles*
- Principle 1: Write clear and specific instructions
- Principle 2: Give the model time to “think”

**Tactic 1: Use delimiters to clearly indicate distinct parts of the input**

Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, :

In [2]:
text = f"""
wait client.connect();
    const database = client.db("your-database-name");
    const collection = database.collection("your-collection-name");

    // Example: Insert a document
    const result = await collection.insertOne();

    // Example: Retrieve documents
    const documents = await collection.find().toArray();
"""
prompt = f"""
Summarize the code delimited by triple backticks \ 
into a single very small sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

The code connects to a database, inserts a document, and retrieves documents from a collection using async/await functions.


**Tactic 2: Ask for a structured output**

JSON, HTML

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along with their authors and genres. 
Provide them in JSON format with the following keys: book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)
     

```xml
<book>
    <book_id>1</book_id>
    <title>The Midnight Garden</title>
    <author>Isabel Waters</author>
    <genre>Fantasy</genre>
</book>
<book>
    <book_id>2</book_id>
    <title>Shadows of the Past</title>
    <author>Michael Blackwood</author>
    <genre>Mystery</genre>
</book>
<book>
    <book_id>3</book_id>
    <title>Whispers in the Wind</title>
    <author>Sarah Nightingale</author>
    <genre>Romance</genre>
</book>
```


**Tactic 3: Ask the model to check whether conditions are satisfied**

In [7]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)
     

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.


In [8]:
text_2 = f"""
The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)
     

Completion for Text 2:
No steps provided.


**Tactic 4: "Few-shot" prompting**

While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [9]:
prompt = f"""
Your task is to answer in a consistent style.

: Teach me about patience.

: The river that carves the deepest valley flows from a modest spring; the \ 
grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

: Resilience is the ability to bounce back from adversity, just like a tree that bends in the wind but does not break. It is the strength to keep moving forward despite challenges and setbacks, like a phoenix rising from the ashes. It is the belief that no matter how many times you fall, you have the power to rise again.


### Principle 2: Give the model time to “think”

**Tactic 1: Specify the steps required to complete a task**

In [10]:

text = f"""
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill go on a quest to fetch water but encounter misfortune along the way, yet they remain adventurous and spirited.

2 - Jack et Jill partent en quête d'eau mais rencontrent des malheurs en chemin, mais ils restent aventureux et pleins d'entrain.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau mais rencontrent des malheurs en chemin, mais ils restent aventureux et pleins d'entrain.",
  "num_names": 2
}


**Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion**

In [11]:

prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need  help working out the financials. 
- Land costs 250 / square foot
- I negotiated a contract for maintenance that will cost me a flat 10 / square foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet. Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is not correct.

The correct total cost for the first year of operations should be:
Total cost = Land cost + Maintenance cost
Total cost = 250x + 10x
Total cost = 260x

Therefore, the correct total cost for the first year of operations as a function of the number of square feet is 260x.


Note that the student's solution is actually not correct.
We can fix this by instructing the model to work out its own solution first.

In [12]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs 250 / square foot
- I negotiated a contract for maintenance that will cost me a flat 10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet. Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet. Costs:
1. Land cost: 250x
2. Maintenance cost: 10x
Total cost: 250x + 10x = 260x

Total cost for the first year of operations as a function of the number of square feet is 260x.

Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


**Model Limitations: Hallucinations**

In [15]:
prompt = f"""
Tell me about director Giorgio Felucci from Italy
"""
response = get_completion(prompt)
print(response)
     

I'm sorry, but there is no information available about a director named Giorgio Felucci from Italy. It is possible that he is not a well-known or established director in the film industry. If you have any specific details or context about Giorgio Felucci, I may be able to provide more information.


## Iterative Prompt Develelopment

**Generate a marketing product description from a product fact sheet**

In [16]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""
     

In [17]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our sleek and stylish mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of furniture that includes filing cabinets, desks, bookcases, meeting tables, and more, allowing you to create a cohesive and sophisticated workspace.

Customize your chair with several options of shell colors and base finishes to match your aesthetic preferences. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. You can also opt for the chair with or without armrests, depending on your comfort needs.

Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair offers both stability and functionality. The dimensions of the chair are as follows: width 53 cm, depth 51 cm, height 80 cm, seat height 44 cm, and seat dep

**Issue 1: The text is too long**

Limit the number of words/sentences/characters.

In [19]:

prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

print(len(response.split()))

Introducing our stylish and versatile mid-century inspired office chair. Available in a variety of colors and finishes, with options for upholstery and armrests. Constructed with a durable aluminum base and adjustable height feature. Perfect for both home and office settings. Made in Italy with high-quality materials.
46


**Issue 2. Text focuses on the wrong details**

Ask it to focus on the aspects that are relevant to the intended audience.

In [20]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair from Italy, part of a mid-century inspired furniture collection. Choose from a variety of shell colors and base finishes, with options for upholstery in fabric or leather. Constructed with quality materials including cast aluminum, nylon coating, and HD36 foam for comfort and durability.


In [21]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stylish and versatile office chair, part of a mid-century inspired collection. Available in various shell colors and base finishes, with options for upholstery in fabric or leather. Constructed with a durable aluminum base and high-density foam seat for comfort. Perfect for home or office use. 

Product IDs: SWC-100, SWC-110


**Issue 3. Description needs a table of dimensions**

Ask it to extract information and organize it in a table.

In [22]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character Product ID in the technical specification.

After the description, include a table that gives the product's dimensions. The table should have two columns.
In the first column include the name of the dimension. In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. Place the description in an element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>This mid-century inspired office chair is the perfect addition to any modern workspace. Constructed with a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action, this chair combines style with functionality. Available in a variety of shell colors and base finishes, as well as options for upholstery, this chair can be customized to fit any aesthetic. Whether for home or business settings, this chair is qualified for contract use, making it a versatile option for any space.</p>
  <p>Constructed with a shell base glider made of cast aluminum with modified nylon PA6/PA66 coating and a seat cushion filled with HD36 foam, this chair is both durable and comfortable. The shell thickness of 10mm ensures stability and longevity, while the Italian craftsmanship guarantees quality.</p>
  <p>Product IDs: SWC-100, SWC-110</p>
</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurements (inches)</th>
  </

**View in HTML**

In [23]:
from IPython.display import display, HTML
     
display(HTML(response))

Dimension,Measurements (inches)
WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""


## Summarizing

**Text to summarize**

In [24]:
prod_review = """
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""

**Summarize with a word/sentence/character limit**

In [26]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site. 

Summarize the review below, delimited by triple backticks, in at most 10 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Soft, cute panda plush, small size, arrived early, daughter loves it.


**Summarize with a focus on shipping and delivery**

In [27]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the Shipping deparmtment. 

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects that mention 
shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The product arrived a day earlier than expected, allowing for personal enjoyment before gifting. Consider offering larger size options for the price paid.


**Summarize with a focus on price and value**

In [28]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy is loved for its softness and cuteness, but may be perceived as slightly overpriced compared to similar options. Arrived early.


**Try "extract" instead of "summarize"**

In [29]:
prompt = f"""
Your task is to extract relevant information from a product review from an ecommerce site to give \ 
feedback to the Shipping department. 

From the review below, delimited by triple quotes extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Feedback: The shipping department is doing well as the product arrived a day earlier than expected, allowing for early enjoyment. It's important to ensure accuracy in delivery estimates.


## Inferring

**Product review text**

In [30]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

**Sentiment (positive/negative)**

In [31]:
prompt = f"""
What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive. The reviewer is satisfied with the lamp, the customer service, and the overall experience with the company.


In [32]:
prompt = f"""
What is the sentiment of the following product review, which is delimited with triple backticks?

Give your answer as a single word, either "positive" or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


**Identify types of emotions**

In [33]:
prompt = f"""
Identify a list of emotions that the writer of the following review is expressing. Include no more than \
five items in the list. Format your answer as a list of lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


**Identify anger**

In [34]:
prompt = f"""
Is the writer of the following review expressing anger? The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


**Extract product and company name from customer reviews**

In [35]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with \
"Item" and "Brand" as the keys. If the information isn't present, use "unknown" as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Item": "lamp",
    "Brand": "Lumina"
}


**Doing multiple tasks at once**

In [36]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys. If the information isn't present, use "unknown" as the value.
Make your response as short as possible. Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


**Infer 5 topics**

In [37]:
story = """
In a recent survey conducted by the government, public sector employees were asked to rate their level 
of satisfaction with the department they work at. The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction 
rating, with only 45 percents of employees indicating they were satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and work towards improving job satisfaction across all departments.
"""

In [38]:
prompt = f"""
Determine five topics that are being discussed in the following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Survey
2. Job satisfaction
3. NASA
4. Social Security Administration
5. Government pledge


In [39]:
print(response.split(sep=','))

['1. Survey\n2. Job satisfaction\n3. NASA\n4. Social Security Administration\n5. Government pledge']


## Transforming

**Translation**

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [40]:
prompt = f"""
Translate the following English text to Ukrainian: ```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)
     

Привіт, я хотів би замовити блендер.


In [41]:
prompt = f"""
Tell me which language this is: ```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This is French.


In [42]:
prompt = f"""
Translate the following  text to French and Spanish and English pirate: ```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: Je veux commander un ballon de basket
Spanish: Quiero ordenar un balón de baloncesto
English Pirate: I be wantin' to order a basketball


In [43]:
prompt = f"""
Translate the following text to Spanish in both the formal and informal forms: 'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


**Universal Translator**

Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [44]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [45]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 불이 켜지지 않는 픽셀이 있어요." 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Korean: 내 Ctrl 키가 고장 났어요 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering
Korean: 내 화면이 깜박거립니다 



**Tone Transformation**

Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [46]:
prompt = f"""
Translate the following from slang to a business letter: 'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the specifications of the standing lamp we discussed. Please see attached. 

Thank you,
Joe


**Format Conversion**

ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [47]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)

In [48]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


**Spellcheck/Grammar check.**

Here are some examples of common grammar and spelling problems and the LLM's response.
To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [49]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found
Their goes my freedom. There going to bring they’re suitcases.

Their goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
No errors found
No errors found.


In [50]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [51]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  </span><span style='color:green;font-weight:700;'>price. </span>It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [52]:
prompt = f"""
proofread and correct this review. Make it more compelling. Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush for my daughter's birthday as she kept borrowing mine from my room. It's not just for kids, adults can appreciate pandas too! She absolutely adores it and carries it everywhere due to its super soft and cute design. However, I did notice one ear is slightly lower than the other, which seems unintentional. Despite this, I found the size to be a bit smaller than expected given the price. I believe there are potentially larger options available for the same cost. On a positive note, the delivery was prompt, arriving a day earlier than anticipated, allowing me to enjoy it briefly before gifting it to my daughter.

## Expanding

**Customize the automated reply to a customer email**

In [53]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal sale for around 70-10 or \
so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about two days.
"""

In [54]:
prompt = f"""
You are a customer service AI assistant. Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for their review.
If the sentiment is negative, apologize and suggest that they can reach out to customer service. 
Make sure to use specific details from the review. Write in a concise and professional tone.
Sign the email as `AI customer agent`. Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your feedback with us. We sincerely apologize for the issues you experienced with the product and the decrease in quality you noticed over time. Your detailed review is greatly appreciated as it helps us understand areas where we can improve.

If you encounter any further issues or have any concerns, please do not hesitate to reach out to our customer service team for assistance. We strive to provide the best possible experience for our customers and your satisfaction is our top priority.

Thank you once again for your valuable feedback.

AI Customer Agent


# Additional Techniques

**Tree of Thoughts (ToT)**

ToT maintains a tree of thoughts, where thoughts represent coherent language sequences that serve as intermediate steps toward solving a problem. This approach enables an LM to self-evaluate the progress through intermediate thoughts made towards solving a problem through a deliberate reasoning process. The LM's ability to generate and evaluate thoughts is then combined with search algorithms (e.g., breadth-first search and depth-first search) to enable systematic exploration of thoughts with lookahead and backtracking.

In [55]:
question = f"""
Here we have a book, 19 cents, a laptop, a bottle, an apple and a sheet of paper. 
Please tell me how to stack them onto each other in a stable manner.
"""

In [56]:
prompt = f"""
Please, answer the following question: ```{question}```
"""
response = get_completion(prompt)
print(response)

To stack the items in a stable manner, you can follow these steps:

1. Place the book flat on the surface as the base.
2. Place the laptop on top of the book, ensuring it is centered and stable.
3. Place the bottle on top of the laptop, making sure it is balanced.
4. Place the apple on top of the bottle, positioning it carefully to avoid rolling off.
5. Finally, place the sheet of paper on top of the apple, making sure it is flat and not hanging over the edges.

By following these steps, you should be able to stack the items in a stable manner.


In [58]:
prompt = f"""
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
When the experts are done with answering, they all go again over the problem and modify their answers.
The question is ```{question}```
"""
response = get_completion(prompt)
print(response)

Expert 1: I would start by stacking the book at the bottom since it is the biggest and most stable item.

Expert 2: I would then place the laptop on top of the book, making sure it is centered to distribute the weight evenly.

Expert 3: Next, I would place the bottle on top of the laptop, using its cylindrical shape to balance it securely.

Expert 1: I think I would then place the sheet of paper on top of the bottle, as it is the lightest item and won't disrupt the balance.

Expert 2: Lastly, I would place the apple on top of the paper, making sure it is positioned in the center to prevent any rolling off. 

After going over the problem again, the experts realize that stacking the apple on top of the paper might not be the most stable option. They decide to revise their answer and place the apple on the bottle instead, using its round shape to prevent it from rolling off.
